# Train/Test/Validation Split

In [5]:
from dsm import datasets
from dsm.dsm_api import DeepSurvivalMachines

import numpy as np

from sklearn.model_selection import ParameterGrid
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc

### Loading the Support Dataset 

In [6]:
x, t, e = datasets.load_dataset('SUPPORT')

### Computing the required quantile of the data 

In [7]:
times = np.quantile(t[e == 1], [0.25, 0.5, 0.75]).tolist()

### Defining the train, test and validation size

In [8]:
data_size = len(x)
tr_size = round(data_size * 0.70)
vl_size = round(data_size * 0.10)
te_size = round(data_size * 0.20)

### Splitting the data into train, test and validation sets

In [9]:
x_train = x[: tr_size]
t_train = t[: tr_size]
e_train = e[: tr_size]

x_test = x[-te_size :]
t_test = t[-te_size :]
e_test = e[-te_size :]

x_val = x[tr_size : tr_size + vl_size]
t_val = t[tr_size : tr_size + vl_size]
e_val = e[tr_size : tr_size + vl_size]

In [10]:
et_train = np.array([(e_train[i], t_train[i]) for i in range(len(e_train))],
                 dtype = [('e', bool), ('t', int)])
et_test = np.array([(e_test[i], t_test[i]) for i in range(len(e_test))],
                 dtype = [('e', bool), ('t', int)])
et_val = np.array([(e_val[i], t_val[i]) for i in range(len(e_val))],
                 dtype = [('e', bool), ('t', int)])

In [11]:
print(et_train.shape, et_test.shape, et_val.shape)

(6374,) (1821,) (910,)


### Defining the parameter grid

In [12]:
param_grid = {'k' : [3, 4, 6],
              'distribution' : ['LogNormal', 'Weibull'],
              'learning_rate' : [ float(1e-4), float(1e-3)],
              'layers' : [ [], [100], [100, 100] ]
             }
params = ParameterGrid(param_grid)

### Validation 

In [30]:
loss = []
for param in params:
    model = DeepSurvivalMachines(k = param['k'],
                                 distribution = param['distribution'],
                                 layers = param['layers'])
    model.fit(x_train, t_train, e_train, learning_rate = param['learning_rate'])
    out_risk = model.predict_risk(x_val, times)
    out_survival = model.predict_survival(x_val, times)

    cis = []
    brs = []
    for i, _ in enumerate(times):
        cis.append(concordance_index_ipcw(et_train, et_val, out_risk[:, i], times[i])[0])

    brs.append(brier_score(et_train, et_val, out_survival, times)[1])
    print("Concordance Index : ", np.mean(cis))
    print("Brier Score : ", np.mean(brs))
    cdauc = []
    for i, _ in enumerate(times):
        cdauc.append(cumulative_dynamic_auc(et_train, et_val, out_risk[:, i], times[i])[0])
    print("Cumulative_dynamic AUC : ", np.mean(cdauc))
    #print(param)
    loss = loss + [[model.compute_nll(x_train, t_train, e_train), param]]
    print("Loss : ", model.compute_nll(x_train, t_train, e_train))

  0%|          | 0/1 [00:00<?, ?it/s]

ParameterDict(  (1): Parameter containing: [torch.DoubleTensor of size 1]) ParameterDict(  (1): Parameter containing: [torch.DoubleTensor of size 1])


100%|██████████| 1/1 [00:00<00:00,  4.05it/s]
/home/kishan/Documents/Projects/deepsurvival/deep_survival/lib/python3.8/site-packages/torch/nn/modules/container.py:550: UserWarning: Setting attributes on ParameterDict is not supported.
  def __setattr__(self, key: Any, value: Any) -> None:
  1%|          | 58/10000 [00:00<00:17, 571.72it/s]

Concordance Index :  0.5169860629297868
Brier Score :  0.2061851100661561
Cumulative_dynamic AUC :  0.5116474076285633
Loss :  2.0041620917118923


  0%|          | 0/1 [00:00<?, ?it/s]

...

Concordance Index :  0.6053654376643375
Brier Score :  0.20453821000498004
Cumulative_dynamic AUC :  0.6214953596409268
Loss :  4.7978640174777984


  0%|          | 0/1 [00:00<?, ?it/s]

ParameterDict(  (1): Parameter containing: [torch.DoubleTensor of size 1]) ParameterDict(  (1): Parameter containing: [torch.DoubleTensor of size 1])


100%|██████████| 1/1 [00:00<00:00,  2.53it/s]

Concordance Index :  0.7024901104239617
Brier Score :  0.18258214715843393
Cumulative_dynamic AUC :  0.7330286501128546
Loss :  4.69341484362596


In [31]:
best_param = min(loss)

### Training  

In [32]:
model = DeepSurvivalMachines(k = best_param[1]['k'],
                                 distribution = best_param[1]['distribution'],
                                 layers = best_param[1]['layers'])
model.fit(x_train, t_train, e_train,
          learning_rate = best_param[1]['learning_rate'])

/home/kishan/Documents/Projects/deepsurvival/deep_survival/lib/python3.8/site-packages/torch/nn/modules/container.py:550: UserWarning: Setting attributes on ParameterDict is not supported.
  def __setattr__(self, key: Any, value: Any) -> None:
  0%|          | 0/1 [00:00<?, ?it/s]

ParameterDict(  (1): Parameter containing: [torch.DoubleTensor of size 1]) ParameterDict(  (1): Parameter containing: [torch.DoubleTensor of size 1])


100%|██████████| 1/1 [00:00<00:00,  2.21it/s]


### Prediction 

In [33]:
out_risk = model.predict_risk(x_test, times)
out_survival = model.predict_survival(x_test, times)

### Evaluation

In [34]:
cis = []
brs = []
for i, _ in enumerate(times):
    cis.append(concordance_index_ipcw(et_train, et_test, out_risk[:, i], times[i])[0])

brs.append(brier_score(et_train, et_test, out_survival, times)[1])
print("Concordance Index:", np.mean(cis))
print("Brier Score:", np.mean(brs))
cdauc = []
for i, _ in enumerate(times):
    cdauc.append(cumulative_dynamic_auc(et_train, et_test, out_risk[:, i], times[i])[0])
print("Cumulative_dynamic AUC ", np.mean(cdauc))

Concordance Index: 0.7012368502533013
Brier Score: 0.17760451012060388
Cumulative_dynamic AUC  0.7302332917015416
